In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 침윤성 암 여부에 따른 이미지 패치화

## 필요 라이브러리 임포트

In [ ]:
import os
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# Install the OpenSlide C library and Python bindings
!apt-get install openslide-tools
!apt-get install python3-openslide

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 5 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (124 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-2_

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
import shutil
import time
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
from PIL import Image
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split

!pip install ipython-autotime

%load_ext autotime
%tensorflow_version 2.x
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.6 MB/s 
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
time: 1.99 s (started: 2022-11-24 04:53:08 +00:00)


In [ ]:
!pwd

/content
time: 163 ms (started: 2022-11-21 06:40:47 +00:00)


In [ ]:
%cd ./drive/MyDrive/최애림/cakd7/m9_딥러닝/3차 프로젝트/5팀(윤정준, 최애림, 진청아, 이정현a)/코드

/content/drive/.shortcut-targets-by-id/1niQkF2oFYkuV2aahMfHbpa3kXs5yrcAd/5팀(윤정준, 최애림, 진청아, 이정현a)/코드
time: 2.14 s (started: 2022-11-23 08:37:45 +00:00)


### 데이터 불러올 경로 지정

In [ ]:
# 경로 지정

slide_dirs = './imgs_tiff/2. train_imgs_tiff'
mask_dirs = './imgs_tiff/2. train_masks_tiff'


slide_paths = [slide_dirs + '/' + slide_img for slide_img in os.listdir(slide_dirs)]
mask_paths = [mask_dirs + '/' + mask_img for mask_img in os.listdir(slide_dirs)]
print(slide_paths[:2])
print(mask_paths[:2])

['./imgs_tiff/2. train_imgs_tiff/BC_01_0015.tiff', './imgs_tiff/2. train_imgs_tiff/BC_01_0397.tiff']
['./imgs_tiff/2. train_masks_tiff/BC_01_0015.tiff', './imgs_tiff/2. train_masks_tiff/BC_01_0397.tiff']
time: 1.83 s (started: 2022-11-22 07:00:22 +00:00)


### 이미지패치화(INVASIVE 학습용 마스크 및 원본) 사용자함수 적용
- 원본이미지, 마스킹이미지에 대해 32*32 이미지 패치로 자른 후, 해당 픽셀값들의 편차가 큰(조직 이미지가 아닌 배경이미지인 경우)를 제외하는 사용자 함수 생성<br>
<sub>참고: https://github.com/longluu/DL-CancerDetection-CAMELYON16</sub>

In [ ]:
def read_slide(slide, x, y, level, width, height, as_float=False):
    # slide: 슬라이드 형식의 이미지 파일
    im = slide.read_region((x,y), level, (width, height))
    # 'RGB'형식으로 변환
    im = im.convert('RGB') # drop the alpha channel
    
    if as_float:
        im = np.asarray(im, dtype = np.float32)
    else:
        im = np.asarray(im)
    
    # 지정된 높이, 너비의 RGB 배열인지 확인
    assert im.shape == (height, width, 3)
    
    return im


# patch_size=100, level=4 는 임의 값들임 
# def make_patch_set(slide_paths, mask_paths, patch_size = 100, level = 4):
def make_patch_set(slide_paths, mask_paths, patch_size = 100):
    
    # 슬라이드 패치와 패치의 라벨값을 저장해줄 빈 리스트 두개 생성 
    slide_patches = []
    tumor_indicator = []
    
    # 아래 값들은 롱루씨가 경험 기반으로 조직과 배경을 구분하기 위해 정한 값들
    threshold_std = 5
    threshold_intensity = 0.2

    #슬라이드 이미지와 해당 마스크 이미지 차례대로 for문 돌려서 패치화 해주기
    for slide_path, mask_path in zip(slide_paths, mask_paths):

        slide_whole = open_slide(slide_path)
        tumor_mask = open_slide(mask_path)
        
        # 전체 이미지 패치 해상도 레벨 확인
        level = slide_whole.level_count - 1


        # 이미지 크기에 맞춰서 정해진 패치 사이즈 (32*32)에 해당하는 step 지정
        n_step_width = tumor_mask.level_dimensions[0][0] // patch_size
        n_step_height = tumor_mask.level_dimensions[0][1] // patch_size
        
        #이건 아직 잘 모르겠음 
        downsample_factor = tumor_mask.level_downsamples[0]  
                
        for step_width in range(n_step_width):
            for step_height in range(n_step_height):

                #우리 CNN에서 maxpooling할 때 처럼 이미지를 아래 창문 크기 사이즈로 옆으로 움직이는 모습을 상상하면 될듯 
                slide_window = [int(patch_size * step_width * downsample_factor), 
                                int(patch_size * step_height * downsample_factor)] 


                #위에 생성해준 read_slide 사용자 함수를 이용하여 패치당 들어가는 픽셀 값들을 배열로 나타내줌

                # Slide 패치들 생성 
                slide_patch = read_slide(slide_whole, 
                                            x = slide_window[0], 
                                            y = slide_window[1], 
                                            level = 0, 
                                            width = patch_size, 
                                            height = patch_size)   
                
                print('slide_patch:', slide_patch)

                # Mask 패치들 생성 
                im_mask = read_slide(tumor_mask, 
                                     x = slide_window[0], 
                                     y = slide_window[1], 
                                     level = 0, 
                                     width = patch_size, 
                                     height = patch_size) 
              
                # Only save those patches that contain cells
                # 패치의 variance가 높다는 뜻은 색상간의 차이가 많다는 것이기 때문에,
                # 즉, 패치가 배경색만 들어있다면, 색상간의 차이가 없다고 이해하면 됨.
                # 아래에서는 패치에 조직세포가 threshold_std 인 0.2 보다 작다면 저장 안해준다, 즉 완전 배경색이 가득한 패치들은 저장 안해줌. 
                if np.mean(np.std(slide_patch, axis = -1)) > threshold_std:
                    slide_patches.append(slide_patch)  
              
                    #종양있는 부분은 픽셀값이 255, 즉 0보다 크기 때문에 tumor indicator 라는 라벨 값이 1이게 된다. 
                    if (29 in im_mask[:,:,0]) & (29 in im_mask[:,:,1]) & (29 in im_mask[:,:,2]):
                        tumor_indicator.append(1)
                    elif (75 in im_mask[:,:,0]) & (75 in im_mask[:,:,1]) & (75 in im_mask[:,:,2]):
                        tumor_indicator.append(1)
                    else:
                        tumor_indicator.append(0)
    # print('patches:', len(slide_patches))

    #순서대로 나열되기 때문에, slide_patches[0]의 라벨값은 tumor_indicator에서 같은 인덱스, tumor_indicator[0] 이게 된다.
    return slide_patches, tumor_indicator

time: 10.7 ms (started: 2022-11-22 07:00:02 +00:00)


In [ ]:
slide_patches, tumor_indicator = make_patch_set(slide_paths, mask_paths, patch_size = 32)
tumor_indicator

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  [241 241 241]
  [241 241 242]]]
slide_patch: [[[241 241 241]
  [241 241 243]
  [241 241 241]
  ...
  [241 242 241]
  [241 241 241]
  [241 241 241]]

 [[241 241 241]
  [241 241 242]
  [241 241 241]
  ...
  [241 242 241]
  [241 241 241]
  [241 241 241]]

 [[241 241 241]
  [241 241 241]
  [241 241 241]
  ...
  [241 242 242]
  [241 242 242]
  [241 241 241]]

 ...

 [[242 241 242]
  [242 241 243]
  [241 241 241]
  ...
  [241 241 241]
  [240 241 241]
  [241 241 241]]

 [[242 241 242]
  [242 241 243]
  [241 241 241]
  ...
  [241 241 241]
  [240 241 241]
  [241 241 241]]

 [[242 241 242]
  [242 241 243]
  [241 241 241]
  ...
  [241 241 241]
  [240 241 241]
  [241 241 241]]]
slide_patch: [[[241 241 241]
  [241 241 241]
  [241 241 241]
  ...
  [241 241 241]
  [241 241 242]
  [241 241 241]]

 [[241 241 241]
  [241 241 241]
  [241 241 241]
  ...
  [241 241 241]
  [241 241 242]
  [241 241 241]]

 [[243 242 242]
  [241 242 242]
  [241 241 241]
  ...
  [240 241 2

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,


time: 36min 11s (started: 2022-11-22 07:00:29 +00:00)


### 저장하기

In [ ]:
len(slide_patches), len(tumor_indicator)

(288724, 144362)

time: 4.48 ms (started: 2022-11-22 07:36:41 +00:00)


In [ ]:
np.save('./array/1122_invasive_indicator.npy', tumor_indicator)
np.save('./array/1122_real_invasive_indicator.npy', slide_patches)

time: 1.73 s (started: 2022-11-22 07:44:22 +00:00)
